<a href="https://colab.research.google.com/github/stellagerantoni/learning-time-series-counterfactuals/blob/main/multivariate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! git clone https://github.com/stellagerantoni/learning-time-series-counterfactuals
 %cd learning-time-series-counterfactuals/

Cloning into 'learning-time-series-counterfactuals'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 318 (delta 126), reused 24 (delta 24), pack-reused 138
Receiving objects: 100% (318/318), 4.39 MiB | 6.30 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/learning-time-series-counterfactuals


In [2]:

!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon[all_extras]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.1/812.1 kB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals

from _composite import ModifiedLatentCF
%cd src
from _vanilla import LatentCF
from help_functions import (ResultWriter, conditional_pad, evaluate,
                            find_best_lr, plot_graphs,
                            reset_seeds, time_series_normalize,
                            time_series_revert, upsample_minority,
                            validity_score)
from keras_models import *

/content/learning-time-series-counterfactuals/src


In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

## **FUNCTIONS**

In [14]:
def load_dataset(dataset):
  X, y, meta_data = load_classification(dataset)
  if dataset == 'Heartbeat':
    pos = 'normal'
    neg = 'abnormal'
    X = X.transpose(0,2,1)
  if dataset == 'SelfRegulationSCP1':
    pos = 'positivity'
    neg = 'negativity'
    X = X.transpose(0,2,1)

  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y,pos_label, neg_label

In [26]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np


def normalize_multivariate(data, n_timesteps, n_features, scaler=None):

    # Then reshape data to have timesteps as rows for normalization
    data_reshaped = data.reshape(-1, n_features)

    if scaler is None:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(data_reshaped)

    normalized = scaler.transform(data_reshaped)

    # Return data reshaped
    data = normalized.reshape(-1, n_timesteps, n_features)
    return data, scaler

def conditional_pad_multivariate(X):
    num_timesteps = X.shape[1]

    if num_timesteps % 4 != 0:
        next_num = (int(num_timesteps / 4) + 1) * 4
        padding_size = next_num - num_timesteps
        X_padded = np.pad(
            X, pad_width=((0, 0), (0, padding_size), (0, 0))
        )

        return X_padded, padding_size

    return X, 0



# **ACTUALL CODE**
datasets available : 'Heartbeat', 'SelfRegulationSCP1'

In [27]:
X,y,pos_label, neg_label = load_dataset('SelfRegulationSCP1')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

 Shape of X =  (561, 896, 6)
 Shape of y =  (561,)
 Meta data =  {'problemname': 'selfregulationscp1', 'timestamps': False, 'missing': False, 'univariate': False, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['negativity', 'positivity']}

 X[:1] = 
[[[23.   19.03 32.19 43.66 30.72 39.09]
  [21.66 19.19 36.81 41.22 31.81 39.53]
  [20.84 21.5  40.16 39.69 31.69 40.59]
  ...
  [21.84 27.03 32.5  50.06 44.69 44.03]
  [22.62 26.78 34.53 50.84 43.69 44.75]
  [21.5  28.94 37.12 50.75 41.88 46.19]]]


In [28]:
# Upsample the minority class

pos_counts = pd.value_counts(y_train)[pos_label]
neg_counts = pd.value_counts(y_train)[neg_label]
print(f"negative_count = {neg_counts}, positive_count = {pos_counts}")

if pos_counts!=neg_counts:
  X_train, y_train = upsample_minority(X_train, y_train, pos_label=pos_label, neg_label=neg_label)
  print(f"Data upsampling performed, current distribution of y: \n{pd.value_counts(y_train)}.")
else:
   print(f"Data upsampling not needed, current distribution of y: \n{pd.value_counts(y_train)}.")


negative_count = 225, positive_count = 223
Data upsampling performed, current distribution of y: 
0    225
1    225
dtype: int64.


In [29]:
n_training,n_timesteps, n_features= X_train.shape

X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")

Data pre-processed, original #timesteps=896, padded #timesteps=896.

min value = -73.84, max value = 175.06
min value normalized = 0.0, max value normalized= 1.0

X_train.shape = (450, 896, 6)
X_train_processed_padded.shape = (450, 896, 6)


In [30]:
y_train_classes = y_train
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

In [11]:
def Classifier(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=1
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    inputs = keras.Input(shape=(n_features, n_timesteps), dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128)(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier

In [42]:
from keras.regularizers import l2
def Classifier(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=1
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    inputs = keras.Input(shape=( n_timesteps,n_features), dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128)(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier
def LSTMFCNClassifier(n_timesteps, n_features, n_output=2, n_LSTM_cells=8, regularization_rate = 0.001):
    # https://github.com/titu1994/LSTM-FCN/blob/master/hyperparameter_search.py
    inputs = keras.Input(shape=( n_features,n_timesteps), dtype="float32")

    x = keras.layers.LSTM(units=n_LSTM_cells, kernel_regularizer=l2(regularization_rate))(inputs)
    x = keras.layers.Dropout(rate=0.8)(x)

    y = keras.layers.Permute((2, 1))(inputs)
    y = keras.layers.Conv1D(64, 8, padding="same", kernel_initializer="he_uniform", kernel_regularizer=l2(regularization_rate))(y)
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.ReLU()(y)

    y = keras.layers.Conv1D(128, 5, padding="same", kernel_initializer="he_uniform", kernel_regularizer=l2(regularization_rate))(y)
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.ReLU()(y)

    y = keras.layers.Conv1D(64, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=l2(regularization_rate))(y)
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.ReLU()(y)

    y = keras.layers.GlobalAveragePooling1D()(y)

    x = keras.layers.concatenate([x, y])

    outputs = keras.layers.Dense(n_output, activation="softmax", kernel_regularizer=l2(regularization_rate))(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier

def ClassifierLSTM(n_timesteps, n_features, extra_lstm_layer=True, n_output=1):
    # Define the model structure - only LSTM layers
    # https://www.kaggle.com/szaitseff/classification-of-time-series-with-lstm-rnn
    inputs = keras.Input(shape=(n_features,n_timesteps), dtype="float32")
    if extra_lstm_layer:
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(
            inputs
        )  # set return_sequences true to feed next LSTM layer
    else:
        x = keras.layers.LSTM(32, activation="tanh", return_sequences=False)(
            inputs
        )  # set return_sequences false to feed dense layer directly
    x = keras.layers.BatchNormalization()(x)
    # x = keras.layers.LSTM(32, activation='tanh', return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    if extra_lstm_layer:
        x = keras.layers.LSTM(16, activation="tanh", return_sequences=False)(x)
        x = keras.layers.BatchNormalization()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier2 = keras.Model(inputs, outputs)

    return classifier2

def Classifier_FCN(input_shape, nb_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation="relu")(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation("relu")(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation("relu")(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(nb_classes, activation="softmax")(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [40]:
X_train_processed_padded = X_train_processed_padded.transpose(0,2,1)
X_test_processed_padded = X_test_processed_padded.transpose(0,2,1)

In [41]:
n_lstmcells = 8

# ## 2. LatentCF models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ## 2.0 LSTM-FCN classifier
###############################################
# ### LSTM-FCN classifier
classifier = LSTMFCNClassifier(
    n_timesteps_padded, n_features, n_output=2, n_LSTM_cells=n_lstmcells
)

optimizer = keras.optimizers.Adam(lr=0.0001)
classifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=30, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = classifier.fit(
    X_train_processed_padded,
    y_train,
    epochs=150,
    batch_size=32,
    shuffle=True,
    verbose=True,
    validation_data=(X_test_processed_padded, y_test),
    callbacks=[early_stopping_accuracy],
)

y_pred = classifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:pos", "True:neg"],
    columns=["Pred:pos", "Pred:neg"],
)
print(f"Confusion matrix: \n{confusion_matrix_df}.")

Training log for LSTM-FCN classifier:
Epoch 1/150
15/15 [==============================] - 8s 89ms/step - loss: 1.0769 - accuracy: 0.7711 - val_loss: 1.2399 - val_accuracy: 0.5044
Epoch 2/150
15/15 [==============================] - 0s 22ms/step - loss: 1.0126 - accuracy: 0.7667 - val_loss: 1.2158 - val_accuracy: 0.5044
Epoch 3/150
15/15 [==============================] - 0s 21ms/step - loss: 0.9620 - accuracy: 0.7911 - val_loss: 1.1884 - val_accuracy: 0.5044
Epoch 4/150
15/15 [==============================] - 0s 23ms/step - loss: 0.9180 - accuracy: 0.7711 - val_loss: 1.1736 - val_accuracy: 0.5044
Epoch 5/150
15/15 [==============================] - 0s 25ms/step - loss: 0.8958 - accuracy: 0.7533 - val_loss: 1.0916 - val_accuracy: 0.5221
Epoch 6/150
15/15 [==============================] - 0s 20ms/step - loss: 0.8629 - accuracy: 0.7644 - val_loss: 1.0618 - val_accuracy: 0.5310
Epoch 7/150
15/15 [==============================] - 0s 18ms/step - loss: 0.8036 - accuracy: 0.7933 - val_loss

In [43]:



reset_seeds()
###############################################
# ### LSTM classifier
classifier = ClassifierLSTM(
    n_timesteps_padded, n_features,extra_lstm_layer=False, n_output=2
)

optimizer = keras.optimizers.Adam(lr=0.001)
classifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=30, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM classifier:")
classifier_history = classifier.fit(
    X_train_processed_padded,
    y_train,
    epochs=150,
    batch_size=42,
    shuffle=True,
    verbose=2,
    validation_data=(X_test_processed_padded, y_test),
    callbacks=[early_stopping_accuracy],
)

y_pred = classifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:pos", "True:neg"],
    columns=["Pred:pos", "Pred:neg"],
)
print(f"Confusion matrix: \n{confusion_matrix_df}.")

Training log for LSTM classifier:
Epoch 1/150
11/11 - 5s - loss: 0.4709 - accuracy: 0.7933 - val_loss: 0.7260 - val_accuracy: 0.5044 - 5s/epoch - 429ms/step
Epoch 2/150
11/11 - 0s - loss: 0.3649 - accuracy: 0.8489 - val_loss: 0.6449 - val_accuracy: 0.5752 - 133ms/epoch - 12ms/step
Epoch 3/150
11/11 - 0s - loss: 0.3523 - accuracy: 0.8600 - val_loss: 0.6244 - val_accuracy: 0.5841 - 143ms/epoch - 13ms/step
Epoch 4/150
11/11 - 0s - loss: 0.3306 - accuracy: 0.8622 - val_loss: 0.6391 - val_accuracy: 0.5487 - 135ms/epoch - 12ms/step
Epoch 5/150
11/11 - 0s - loss: 0.3197 - accuracy: 0.8556 - val_loss: 0.6086 - val_accuracy: 0.5929 - 131ms/epoch - 12ms/step
Epoch 6/150
11/11 - 0s - loss: 0.2936 - accuracy: 0.8667 - val_loss: 0.5687 - val_accuracy: 0.7168 - 126ms/epoch - 11ms/step
Epoch 7/150
11/11 - 0s - loss: 0.2735 - accuracy: 0.8911 - val_loss: 0.5807 - val_accuracy: 0.6726 - 120ms/epoch - 11ms/step
Epoch 8/150
11/11 - 0s - loss: 0.2790 - accuracy: 0.8889 - val_loss: 0.5920 - val_accuracy: 0

In [ ]:
shallow_cnn = False
# ### 1dCNN classifier
if shallow_cnn == True:
    print(f"Check shallow_cnn argument={shallow_cnn}, use the shallow structure.")
    classifier = Classifier(n_timesteps_padded, n_features) # shallow CNN for small data size
else:
    classifier = Classifier(n_timesteps_padded, n_features, n_conv_layers=3, add_dense_layer=False) # deeper CNN layers for data with larger size

optimizer = keras.optimizers.Adam(lr=0.0001)
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
# Train the model
reset_seeds()
classifier_history = classifier.fit(X_train_processed_padded,
        y_train_classes,
        epochs=150,
        batch_size=32,
        shuffle=True,
        verbose=True,
        validation_data=(X_test_processed_padded, y_test_classes),
        callbacks=[early_stopping_accuracy])


y_pred = classifier.predict(X_test_processed_padded)
# y_pred_classes = np.array([1 if pred > 0.5 else 0 for pred in y_pred])
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"1dCNN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
        index=['True:pos', 'True:neg'],
        columns=['Pred:pos', 'Pred:neg']
    )
confusion_matrix_df

Epoch 1/150
15/15 [==============================] - 3s 35ms/step - loss: 0.5416 - accuracy: 0.7622 - val_loss: 0.6624 - val_accuracy: 0.5133
Epoch 2/150
15/15 [==============================] - 0s 16ms/step - loss: 0.4217 - accuracy: 0.8133 - val_loss: 0.6622 - val_accuracy: 0.5221
Epoch 3/150
15/15 [==============================] - 0s 15ms/step - loss: 0.3188 - accuracy: 0.8711 - val_loss: 0.6371 - val_accuracy: 0.5487
Epoch 4/150
15/15 [==============================] - 0s 14ms/step - loss: 0.3593 - accuracy: 0.8378 - val_loss: 0.6095 - val_accuracy: 0.5929
Epoch 5/150
15/15 [==============================] - 0s 16ms/step - loss: 0.3774 - accuracy: 0.8511 - val_loss: 1.6900 - val_accuracy: 0.5133
Epoch 6/150
15/15 [==============================] - 0s 15ms/step - loss: 0.3351 - accuracy: 0.8489 - val_loss: 0.5945 - val_accuracy: 0.6372
Epoch 7/150
15/15 [==============================] - 0s 15ms/step - loss: 0.2872 - accuracy: 0.8822 - val_loss: 0.5388 - val_accuracy: 0.7168
Epoch 

In [ ]:
def Autoencoder(n_timesteps, n_features):
    # Define encoder and decoder structure
    def Encoder(input):
        x = keras.layers.Conv1D(
            filters=64, kernel_size=3, activation="relu", padding="same"
        )(input)
        x = keras.layers.MaxPool1D(pool_size=2, padding="same")(x)
        x = keras.layers.Conv1D(
            filters=32, kernel_size=3, activation="relu", padding="same"
        )(x)
        x = keras.layers.MaxPool1D(pool_size=2, padding="same")(x)
        return x

    def Decoder(input):
        x = keras.layers.Conv1D(
            filters=32, kernel_size=3, activation="relu", padding="same"
        )(input)
        x = keras.layers.UpSampling1D(size=2)(x)
        x = keras.layers.Conv1D(
            filters=64, kernel_size=3, activation="relu", padding="same"
        )(x)
        # x = keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu")(x)
        x = keras.layers.UpSampling1D(size=2)(x)
        x = keras.layers.Conv1D(
            filters=1, kernel_size=3, activation="linear", padding="same"
        )(x)
        return x

    # Define the AE model
    orig_input = keras.Input(shape=(n_timesteps,n_features))
    autoencoder = keras.Model(inputs=orig_input, outputs=Decoder(Encoder(orig_input)))

    return autoencoder

def AutoencoderLSTM(n_timesteps, n_features):
    # Define encoder and decoder structure
    # structure from medium post: https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
    def EncoderLSTM(input):
        # x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(input)
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(input)
        # encoded = keras.layers.LSTM(32, activation='relu', return_sequences=False)(x)
        encoded = keras.layers.LSTM(32, activation="tanh", return_sequences=False)(x)
        return encoded

    def DecoderLSTM(encoded):
        x = keras.layers.RepeatVector(n_timesteps)(encoded)
        # x = keras.layers.LSTM(32, activation='relu', return_sequences=True)(x)
        x = keras.layers.LSTM(32, activation="tanh", return_sequences=True)(x)
        # x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(x)
        x = keras.layers.LSTM(64, activation="tanh", return_sequences=True)(x)
        decoded = keras.layers.TimeDistributed(
            keras.layers.Dense(n_features, activation="sigmoid")
        )(x)
        return decoded

    # Define the AE model
    orig_input2 = keras.Input(shape=( n_timesteps,n_features))

    autoencoder2 = keras.Model(
        inputs=orig_input2, outputs=DecoderLSTM(EncoderLSTM(orig_input2))
    )

    return autoencoder2


In [ ]:
X_train_processed_padded.shape

(450, 896, 6)

In [ ]:
X_test_processed_padded.shape

(113, 896, 6)

In [ ]:
X_train_processed_padded = np.transpose(X_train_processed_padded, (0, 2, 1))

In [ ]:
X_test_processed_padded = np.transpose(X_test_processed_padded, (0, 2, 1))

In [ ]:
reset_seeds()
# ### 1dCNN autoencoder
autoencoder = Autoencoder(n_timesteps_padded, n_features)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train_processed_padded,
    X_train_processed_padded,
    epochs=50,
    batch_size=32,
    shuffle=True,
    verbose=2,
    validation_data=(X_test_processed_padded, X_test_processed_padded),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")

Training log for 1dCNN autoencoder:
Epoch 1/50
15/15 - 7s - loss: 0.0643 - val_loss: 0.0138 - 7s/epoch - 471ms/step
Epoch 2/50
15/15 - 3s - loss: 0.0045 - val_loss: 0.0013 - 3s/epoch - 192ms/step
Epoch 3/50
15/15 - 3s - loss: 0.0014 - val_loss: 0.0012 - 3s/epoch - 173ms/step
Epoch 4/50
15/15 - 2s - loss: 0.0011 - val_loss: 9.0219e-04 - 2s/epoch - 110ms/step
Epoch 5/50
15/15 - 2s - loss: 8.1441e-04 - val_loss: 6.6226e-04 - 2s/epoch - 109ms/step
Epoch 6/50
15/15 - 2s - loss: 6.2639e-04 - val_loss: 6.2277e-04 - 2s/epoch - 108ms/step
Epoch 7/50
15/15 - 2s - loss: 5.9381e-04 - val_loss: 5.9856e-04 - 2s/epoch - 107ms/step
Epoch 8/50
15/15 - 2s - loss: 5.7754e-04 - val_loss: 5.8973e-04 - 2s/epoch - 107ms/step
Epoch 9/50
15/15 - 2s - loss: 5.7019e-04 - val_loss: 5.8374e-04 - 2s/epoch - 118ms/step
Epoch 10/50
15/15 - 3s - loss: 5.6577e-04 - val_loss: 5.8037e-04 - 3s/epoch - 203ms/step
1dCNN autoencoder trained, with validation loss: 0.0005803668755106628.


In [ ]:
reset_seeds()
# ### 1dCNN autoencoder
autoencoderLSTM = AutoencoderLSTM(n_timesteps_padded, n_features)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoderLSTM.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoderLSTM.fit(
    X_train_processed_padded,
    X_train_processed_padded,
    epochs=50,
    batch_size=32,
    shuffle=True,
    verbose=2,
    validation_data=(X_test_processed_padded, X_test_processed_padded),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")

Training log for 1dCNN autoencoder:
Epoch 1/50
15/15 - 45s - loss: 0.0183 - val_loss: 0.0093 - 45s/epoch - 3s/step
Epoch 2/50
15/15 - 33s - loss: 0.0106 - val_loss: 0.0085 - 33s/epoch - 2s/step
Epoch 3/50
15/15 - 33s - loss: 0.0101 - val_loss: 0.0077 - 33s/epoch - 2s/step
Epoch 4/50
15/15 - 33s - loss: 0.0080 - val_loss: 0.0048 - 33s/epoch - 2s/step
Epoch 5/50
15/15 - 33s - loss: 0.0089 - val_loss: 0.0082 - 33s/epoch - 2s/step
Epoch 6/50
15/15 - 33s - loss: 0.0069 - val_loss: 0.0047 - 33s/epoch - 2s/step
Epoch 7/50
15/15 - 33s - loss: 0.0051 - val_loss: 0.0041 - 33s/epoch - 2s/step
Epoch 8/50
15/15 - 34s - loss: 0.0046 - val_loss: 0.0038 - 34s/epoch - 2s/step
Epoch 9/50
15/15 - 33s - loss: 0.0045 - val_loss: 0.0034 - 33s/epoch - 2s/step
Epoch 10/50
15/15 - 33s - loss: 0.0039 - val_loss: 0.0033 - 33s/epoch - 2s/step
Epoch 11/50
15/15 - 33s - loss: 0.0037 - val_loss: 0.0060 - 33s/epoch - 2s/step
Epoch 12/50
15/15 - 34s - loss: 0.0055 - val_loss: 0.0042 - 34s/epoch - 2s/step
Epoch 13/50
1